In [1]:
import numpy as np
#from skimage import io
#from skimage import transform
import cv2
import matplotlib.pyplot as plt
import os
import torch
import torchvision
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import transforms
from torchvision.utils import make_grid
import math
import time
import pandas as pd

In [2]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(  
            nn.Conv2d(
                in_channels=1,      # input height
                out_channels=16,    # n_filters
                kernel_size=5,      # filter size
                stride=1,           # filter movement/step
                padding=2,     
            ),      
            nn.ReLU(),    # activation
            nn.MaxPool2d(kernel_size=2),    
        )
        self.conv2 = nn.Sequential(  
            nn.Conv2d(16, 32, 7, 2, 2),  
            nn.ReLU(),  # activation
            nn.MaxPool2d(2),  
        )
        self.conv3 = nn.Sequential(  
            nn.Conv2d(32, 32, 5, 1, 2),  
            nn.ReLU(),  # activation
            nn.MaxPool2d(2),  
        )
        
        self.conv4 = nn.Sequential(  
            nn.Conv2d(32, 32, 3, 1, 2),  
            nn.ReLU(),  # activation

        )
        self.conv5 = nn.Sequential(  
            nn.Conv2d(32, 64, 3, 1, 2),  
            nn.ReLU(),  # activation
 
        )
        self.conv6 = nn.Sequential(  
            nn.Conv2d(64, 32, 3, 1, 2),  
            nn.ReLU(),  # activation
  
        )
        self.conv7 = nn.Sequential(  
            nn.Conv2d(32, 32, 3, 1, 2),  
            nn.ReLU(),  # activation
  
        )
        self.conv8 = nn.Sequential(  
            nn.Conv2d(32, 32, 3, 1, 2),  
            nn.ReLU(),  # activation
  
        )
        self.conv9 = nn.Sequential(  
            nn.Conv2d(32, 16, 3, 1, 2),  
            nn.ReLU(),  # activation
  
        )
        self.conv10 = nn.Sequential(  
            nn.Conv2d(16, 8, 3, 1, 2),  
            nn.ReLU(),  # activation

        )
        
        self.linear1 = nn.Linear(8192, 4096)   
        self.linear2 = nn.Linear(4096, 2048)   
        self.linear3 = nn.Linear(2048, 512)
        self.linear4 = nn.Linear(512, 128)
        self.linear5 = nn.Linear(128, 32)
        self.linear6 = nn.Linear(32, 22)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = self.conv8(x)
        x = self.conv9(x)
        x = self.conv10(x)
        x = x.view(x.size(0), -1)   
        x = self.linear1(x)
        x = self.linear2(x)
        x = self.linear3(x)
        x = self.linear4(x)
        x = self.linear5(x)
        output = self.linear6(x)
        return output        

In [3]:
print("before")
EPOCH = 200           
BATCH_SIZE = 50
LR = 0.0001  
checkpoint = torch.load('24_cnn.pth')
cnn = CNN()
cnn.load_state_dict(checkpoint['net'])

optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)   # optimize all cnn parameters
optimizer.load_state_dict(checkpoint['optimizer'])

loss_func = nn.MSELoss()   # the target label is not one-hotted

before


In [4]:
def alpha2white_opencv2(img):
    sp=img.shape
    width=sp[0]
    height=sp[1]
    for yh in range(height):
        for xw in range(width):
            color_d=img[xw,yh]
            if(color_d[3]==0):
                img[xw,yh]=[255,255,255,255]
    return img

def search_action(list1,action_list,num):
    distance=[]
    for i in range(0,num):
        dis22=0
        for j in range(0,22):
            dis=abs(list1[j]-action_list.iloc[i][j+1])
            dis22=dis22+dis        
        distance.append(dis22)
    min_index=distance.index(min(distance))+1
    return min_index

def black(img):
    for jj in range (300):
        for kk in  range(300):                
            if img[jj,kk]>235:
                img[jj,kk]=255
            else:
                img[jj,kk]=0
    return img

In [ ]:
cnn.eval()
action_list=pd.read_csv("./data_set/action_data.csv")
manga_list=pd.read_csv("./data_set/manga_data.csv")
gans_list=pd.read_csv("./data_set/gan_sketch_data2.csv")
ganm_list=pd.read_csv("./data_set/gan_manga_data2.csv")
print("running")

while True:
    image = cv2.imread("input/2.png",-1)
    if np.shape(image) == ():
        time.sleep(2)
        pass
    else:
        image=alpha2white_opencv2(image)
        cv2.imwrite("input/1.png",image)
        image = cv2.imread("input/1.png",cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image,(300,300),interpolation=cv2.INTER_CUBIC)

        image = black(image)
        image_n = torch.from_numpy(image)
        b_x = image_n.float().unsqueeze(0).unsqueeze(0)
        output = cnn(b_x).int().tolist()

        list1=[]
        for i in range(0,22,2):
            list1.append(output[0][i])
            list1.append(output[0][i+1])
        print(list1)
        
        #action output for 1.png
        action_index=search_action(list1,action_list,332)
        if action_index<=166: 
            action_image = cv2.imread("./data_set/action/"+str(action_index)+".png",cv2.IMREAD_GRAYSCALE)
        if action_index>166:        
            action_image = cv2.imread("./data_set/action/"+str(action_index-166)+".png",cv2.IMREAD_GRAYSCALE)
            action_image = cv2.flip(action_image, 1)
        cv2.imwrite("output/1.png",action_image)
                
        #action output for 2.png
        action_index=search_action(list1,manga_list,540)
        if action_index<=270: 
            action_image = cv2.imread("./data_set/manga/"+str(action_index)+".jpg",cv2.IMREAD_GRAYSCALE)
        if action_index>270:
            action_image = cv2.imread("./data_set/manga/"+str(action_index-270)+".jpg",cv2.IMREAD_GRAYSCALE)
            action_image = cv2.flip(action_image, 1)
        cv2.imwrite("output/2.png",action_image)
               
        #action output for 3.png
        action_index=search_action(list1,gans_list,728)
        if action_index<=364: 
            action_image = cv2.imread("./data_set/gan_sketch/"+str(action_index)+".png",cv2.IMREAD_GRAYSCALE)
        if action_index>364:      
            action_image = cv2.imread("./data_set/gan_sketch/"+str(action_index-364)+".png",cv2.IMREAD_GRAYSCALE)
            action_image = cv2.flip(action_image, 1)
        cv2.imwrite("output/3.png",action_image)
                
        #action output for 4.png
        action_index=search_action(list1,ganm_list,1208)
        if action_index<=604: 
            action_image = cv2.imread("./data_set/gan_manga/"+str(action_index)+".png",cv2.IMREAD_GRAYSCALE)
        if action_index>604:      
            action_image = cv2.imread("./data_set/gan_manga/"+str(action_index-604)+".png",cv2.IMREAD_GRAYSCALE)
            action_image = cv2.flip(action_image, 1)
        cv2.imwrite("output/4.png",action_image)
        
        os.remove("input/2.png")
        print(action_index)
        print("waiting")

     
print("Process off")

running
[149, 87, 152, 130, 103, 111, 77, 88, 196, 113, 184, 140, 155, 174, 129, 208, 121, 243, 194, 202, 217, 248]
656
waiting
[148, 66, 156, 103, 132, 141, 110, 141, 181, 153, 146, 139, 165, 142, 177, 187, 136, 139, 156, 187, 109, 215]
418
waiting
[146, 69, 149, 114, 118, 140, 146, 132, 180, 138, 155, 112, 153, 161, 147, 208, 128, 265, 169, 202, 155, 241]
103
waiting


In [ ]:
''''
http://drawstick.ngrok.io/